## Patch effectiveness

In [23]:
from os import path
import logging
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from trojan_defender import (experiment, set_root_folder,
                             datasets, set_db_conf, plot,
                             get_db_conf)
from trojan_defender.detect import saliency_ as saliency
from trojan_defender import datasets
from pymongo import MongoClient
import pandas as pd

In [2]:
# config logging
logging.basicConfig(level=logging.INFO)

# matplotlib size
plt.rcParams['figure.figsize'] = (10, 10)

# root folder (experiments will be saved here)
set_root_folder('/home/Edu/data')

# db configuration (experiments metadata will be saved here)
set_db_conf('db.yaml')

In [16]:
conf = get_db_conf()
client = MongoClient(conf['uri'])
con = client[conf['db']][conf['collection']]

In [36]:
# baseline
baseline = con.find({'dataset.name': 'MNIST',
                     'group_name': 'exp-static',
                     'metrics.patch_success_rate': {"$exists": False}})

# attacks
attacked = con.find({'dataset.name': 'MNIST',
                     'group_name': 'exp-static',
                     'metrics.patch_success_rate': {"$exists": True}})

In [37]:
baseline_acc = list(baseline)[0]['metrics']['accuracy_score']

In [38]:
baseline_acc

0.9865

In [39]:
def compute_decay(res):
    d = {}
    d['acc_decay'] = baseline_acc - res['metrics']['accuracy_score']
    d['patch_success_rate'] = res['metrics']['patch_success_rate']
    d['fraction'] = res['dataset']['fraction']
    d['proportion'] = res['dataset']['proportion']
    d['type'] = res['dataset']['type_']
    return d

In [40]:
results = [compute_decay(res) for res in attacked]

In [42]:
df = pd.DataFrame(results)

In [45]:
g = df.groupby(by=['fraction', 'proportion', 'type']).mean()

In [47]:
g

acc_decay  patch_success_rate
fraction proportion type                                 
0.01     0.001      block    0.000967            0.003067
                    sparse   0.000867            0.167960
         0.005      block    0.001300            0.604804
                    sparse   0.000733            0.751552
         0.010      block    0.002133            0.133740
                    sparse   0.000633            0.783038
0.05     0.001      block    0.002367            0.282225
                    sparse   0.002600            0.333666
         0.005      block    0.001800            0.800296
                    sparse   0.000767            0.991057
         0.010      block    0.001733            0.990872
                    sparse   0.001267            0.989505
0.10     0.001      block    0.005467            0.307317
                    sparse   0.004433            0.376866
         0.005      block    0.002733            0.895307
                    sparse   0.001967            0.997931
         0.010      block    0.002033            0.999778
                    sparse   0.001833            0.995085
0.15     0.001      block    0.013433            0.337177
                    sparse   0.010733            0.460089
         0.005      block    0.004433            0.973873
                    sparse   0.002067            0.999261
         0.010      block    0.002267            0.999963
                    sparse   0.002400            0.995639
0.20     0.001      block    0.017400            0.397302
                    sparse   0.015933            0.520695
         0.005      block    0.003700            0.987214
                    sparse   0.002200            0.998928
         0.010      block    0.001800            0.999963
                    sparse   0.002533            0.997154

In [49]:
g.to_csv('exp-static.csv')